### TP Final de Aprendizaje Automático
El dataset a usar es **Employee Productivity and Satisfaction HR Data**, tomado de [Kaggle](https://www.kaggle.com/datasets/adityaab1407/employee-productivity-and-satisfaction-hr-data).

Este dataset recopila información respecto a la productividad y desempeño de empleados, teniendo en cuenta algunas características.

In [1]:
import numpy as np # manejo de arreglos, álgebra lineal
import pandas as pd # pandas para procesamiento de archivos

dataset = pd.read_csv('hr_dashboard_data.csv')


# Veamos que tenemos
dataset.head()


,Name,Age,Gender,Projects Completed,Productivity (%),Satisfaction Rate (%),Feedback Score,Department,Position,Joining Date,Salary
0,Douglas Lindsey,25,Male,11,57,25,4.7,Marketing,Analyst,Jan-20,63596
1,Anthony Roberson,59,Female,19,55,76,2.8,IT,Manager,Jan-99,112540
2,Thomas Miller,30,Male,8,87,10,2.4,IT,Analyst,Jan-17,66292
3,Joshua Lewis,26,Female,1,53,4,1.4,Marketing,Intern,Jan-22,38303
4,Stephanie Bailey,43,Male,14,3,9,4.5,IT,Team Lead,Jan-05,101133


# 1. Análisis inicial del dataset

Vemos que tenemos las siguientes variables:
- nombre (insignificante)
- edad
- género (hay que pasarlo a otro tipo de variable, por ej. bool)
- proyectos completados
- % de productividad
- % de satisfacción
- feedback (de 1 a 5)
- departamento (deberíamos pasarlo a variables dummy)
- posición (deberíamos pasarlo a variable dummy)
- fecha de inicio (deberíamos pasarlo a números para que sea más fácil el análisis)
- salario

Vamos a limpiar un poco el dataset...

In [ ]:
dataset = 